##### Importando libs

In [1]:
# import numpy as np
# import pandas as pd
# import logging
# import pickle
# import os

##### Declarando paths

In [2]:
# root_path = os.getenv('HOST_PATH')
# parquets_path = f"{root_path}/artifacts/parquets"
# validacao_target_file = "validacao.parquet"
# users_target_file = "usuarios_test_preprocessados.parquet"
# articles_target_file = "artigos_test_preprocessados.parquet"

##### Carregando Parquets

In [3]:
# df_validacao = pd.read_parquet(f"{parquets_path}/{validacao_target_file}")
# df_users = pd.read_parquet(f"{parquets_path}/{users_target_file}")
# df_articles = pd.read_parquet(f"{parquets_path}/{articles_target_file}")

##### Carregando modelo

In [4]:
# with open(f"{root_path}/artifacts/models/content_model.pkl", "rb") as file:
#   content_model = pickle.load(file)
  
# with open(f"{root_path}/artifacts/models/collaborative_model.pkl", "rb") as file:
#   collaborative_model = pickle.load(file)
  

In [5]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import ast
import os

# Definir caminhos
root_path = os.getenv("HOST_PATH")
parquets_path = f"{root_path}/artifacts/parquets"
models_path = f"{root_path}/artifacts/models"
validation_file = "validacao.parquet"

# Carregar modelos treinados
with open(f"{models_path}/content_model.pkl", "rb") as f:
    content_model = pickle.load(f)

with open(f"{models_path}/collaborative_model.pkl", "rb") as f:
    collaborative_model = pickle.load(f)

# Carregar interações treinadas para referência
train_ratings = pd.read_parquet(f"{parquets_path}/interactions.parquet")

# Carregar dados de validação
df_validation = pd.read_parquet(f"{parquets_path}/{validation_file}")

In [6]:
print(type(df_validation["history"].iloc[0])) 

<class 'numpy.ndarray'>


In [7]:

df_validation["history"] = df_validation["history"].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

# Criar matriz usuário-item baseada no conjunto de treino
user_item_matrix = train_ratings.pivot(index="userId", columns="itemId", values="rating").fillna(0)
sparse_matrix = csr_matrix(user_item_matrix)

# Criar um mapeamento de itemId -> índice na matriz
item_index_map = {item: idx for idx, item in enumerate(user_item_matrix.columns)}

# Função para prever ratings usando modelo colaborativo
def predict_collaborative(user_id, item_id):
    if user_id not in user_item_matrix.index or item_id not in user_item_matrix.columns:
        return 0  # Retornar zero para novos usuários/itens

    user_vector = user_item_matrix.loc[user_id].values.reshape(1, -1)
    prediction = collaborative_model.transform(user_vector).dot(collaborative_model.components_[:, user_item_matrix.columns.get_loc(item_id)])
    return prediction[0]

# Função para prever rankings baseados no modelo de conteúdo
def predict_content_based(item_id):
    if item_id not in item_index_map:
        return np.zeros(len(user_item_matrix.columns))  # Retorna vetor de zeros se o item não for encontrado
    
    item_idx = item_index_map[item_id]
    if item_idx >= content_model.shape[0]:  # Verifica se o índice está dentro dos limites da matriz
        return np.zeros(len(user_item_matrix.columns))

    return content_model[item_idx]  # Retorna as similaridades com outros itens

# Criar DataFrame para armazenar previsões e valores reais
results = []

for _, user in df_validation.iterrows():
    user_id = user["userId"]
    history = user["history"]

    for item_id in history:
        actual_rating = train_ratings[(train_ratings["userId"] == user_id) & (train_ratings["itemId"] == item_id)]["rating"].values
        actual_rating = actual_rating[0] if len(actual_rating) > 0 else 0

        predicted_collaborative = predict_collaborative(user_id, item_id)
        predicted_content = np.mean(predict_content_based(item_id))

        results.append({
            "userId": user_id,
            "itemId": item_id,
            "actual": actual_rating,
            "predicted_collaborative": predicted_collaborative,
            "predicted_content": predicted_content
        })

df_results = pd.DataFrame(results)

# Calcular métricas de erro para cada modelo
rmse_collaborative = np.sqrt(mean_squared_error(df_results["actual"], df_results["predicted_collaborative"]))
rmse_content = np.sqrt(mean_squared_error(df_results["actual"], df_results["predicted_content"]))

print(f"RMSE (Modelo Colaborativo): {rmse_collaborative:.4f}")
print(f"RMSE (Modelo Baseado em Conteúdo): {rmse_content:.4f}")


KeyboardInterrupt: 

In [ ]:
print(f"Usuários na validação: {df_validation['userId'].nunique()}")
print(f"Usuários no treino: {train_ratings['userId'].nunique()}")
print(f"Itens na validação: {df_validation['history'].explode().nunique()}")
print(f"Itens no treino: {train_ratings['itemId'].nunique()}")


Usuários na validação: 112184
Usuários no treino: 7000
Itens na validação: 36341
Itens no treino: 24118


In [ ]:
user_id = train_ratings["userId"].iloc[0]
item_id = train_ratings["itemId"].iloc[0]
print(f"Previsão para usuário {user_id} e item {item_id}: {predict_collaborative(user_id, item_id)}")


Previsão para usuário a50f16d51820754a3db8281180950c3619c2b5a154926cb383de0fd023404756 e item 25d54fe7-248b-4e1d-89a1-20b36aba6159: 1.967555243859656e-05
